In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler  


In [2]:
data = pd.read_excel('data-stiffness_SS.xlsx') 

# remove rows with null value
data = data.dropna(how='any',axis=0) 

# select features and target variable
features = data.iloc[:, :-2]  # all columns except the last two

#target = data.iloc[:, -2:-1]     # stiffness
target = data.iloc[:, -1]     # specific stiffness



X_train_full,X_test,y_train_full ,y_test=train_test_split(features, target , test_size = 0.2,random_state=0, shuffle=True) 
X_train, X_valid, y_train, y_valid =train_test_split(X_train_full, y_train_full ,test_size = 0.2,random_state=0, shuffle=True) # 5624 train


In [3]:
# Feature scaling
scaler = StandardScaler()  

X_train = scaler.fit_transform(X_train)  # numpy.ndarray
X_valid = scaler.transform(X_valid)      # numpy.ndarray
X_test = scaler.transform(X_test)        # numpy.ndarray

In [4]:
#It builds and returns a feedforward neural network (a Sequential model) for regression tasks. 
#The model is customizable via its arguments, such as the number of hidden layers, the number of neurons in each layer, 
#and the learning rate for optimization.

def build_model(n_hidden , n_neurons, learning_rate, shape, active_fun):      # shape is the number of features
    model = keras.models.Sequential()    #each layer has exactly one input and one output
    model.add(keras.layers.InputLayer(shape=shape))     #Adds an input layer to the model 
    for layer in range(n_hidden):       #iterates over the number of hidden_layers
        model.add(keras.layers.Dense(n_neurons , activation=active_fun))  #Adds a dense (fully connected) layer with n_neurons neurons.
    model.add(keras.layers.Dense(1))   #Adds the output layer with a single neuron and no activation function.
    optimizer= keras.optimizers.Adam(learning_rate=learning_rate)  
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [5]:
from scikeras.wrappers import KerasRegressor

keras_reg = KerasRegressor(
    build_model,
    n_hidden=1,
    n_neurons=30,
    learning_rate=3e-3,
    shape=[9],
    active_fun="relu",
    epochs=10,  
    batch_size=32,  
    verbose=0  
)

In [6]:
# getting callbacks

early_stopping = keras.callbacks.EarlyStopping(patience=10,
                                              restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("My_trained_model.keras",
                                                     save_best_only=True,
                                                     monitor='val_loss',
                                                     verbose=2)

In [7]:
keras_reg.fit(X_train , y_train, epochs=10,
             validation_data=(X_valid, y_valid),
             callbacks=[model_checkpoint, early_stopping])

KerasRegressor(
	model=<function build_model at 0x0000023A6EC24A60>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	n_hidden=1
	n_neurons=30
	learning_rate=0.003
	shape=[9]
	active_fun=relu
)

In [8]:
mse_test = keras_reg.score(X_test,y_test)
print(mse_test)

0.833457480141795


In [9]:
y_pred= keras_reg.predict(X_test)

In [10]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


In [11]:
param_distribs = {
    "n_hidden":[3,4],
    "n_neurons":np.arange(30,90)              .tolist(),
    #"learning_rate": reciprocal(3e-3,3e-2)    .rvs(10).tolist(),
    "active_fun":["relu","tanh"],
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=35,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END ..........active_fun=relu, n_hidden=4, n_neurons=69; total time=   4.9s
[CV] END ..........active_fun=relu, n_hidden=4, n_neurons=69; total time=   4.8s
[CV] END ..........active_fun=relu, n_hidden=4, n_neurons=69; total time=   4.7s
[CV] END ..........active_fun=relu, n_hidden=3, n_neurons=47; total time=   4.6s
[CV] END ..........active_fun=relu, n_hidden=3, n_neurons=47; total time=   4.5s
[CV] END ..........active_fun=relu, n_hidden=3, n_neurons=47; total time=   4.5s
[CV] END ..........active_fun=tanh, n_hidden=4, n_neurons=52; total time=   4.7s
[CV] END ..........active_fun=tanh, n_hidden=4, n_neurons=52; total time=   4.7s
[CV] END ..........active_fun=tanh, n_hidden=4, n_neurons=52; total time=   4.7s
[CV] END ..........active_fun=tanh, n_hidden=4, n_neurons=55; total time=   5.0s
[CV] END ..........active_fun=tanh, n_hidden=4, n_neurons=55; total time=   4.7s
[CV] END ..........active_fun=tanh, n_hidden=4, 

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(active_fun='relu', batch_size=32, epochs=10, learning_rate=0.003, model=<function build_model at 0x0000023A6EC24A60>, n_hidden=1, n_neurons=30, shape=[9], verbose=0),
                   n_iter=20,
                   param_distributions={'active_fun': ['relu', 'tanh'],
                                        'n_hidden': [3, 4],
                                        'n_neurons': [30, 31, 32, 33, 34, 35,
                                                      36, 37, 38, 39, 40, 41,
                                                      42, 43, 44, 45, 46, 47,
                                                      48, 49, 50, 51, 52, 53,
                                                      54, 55, 56, 57, 58, 59, ...]},
                   verbose=2)

In [12]:
rnd_search_cv.best_params_

{'n_neurons': 52, 'n_hidden': 4, 'active_fun': 'tanh'}

In [13]:
rnd_search_cv.best_score_

0.9630316149464404

In [34]:
best=rnd_search_cv.best_estimator_
best


KerasRegressor(
	model=<function build_model at 0x00000214E768BD30>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	n_hidden=3
	n_neurons=77
	learning_rate=0.008002101701145322
	shape=[9]
	active_fun=relu
)

In [22]:
my_model=keras_reg = KerasRegressor(
    build_model,
    n_hidden=rnd_search_cv.best_params_["n_hidden"],
    n_neurons=rnd_search_cv.best_params_["n_neurons"],
    learning_rate=rnd_search_cv.best_params_["learning_rate"],
    shape=[9],
    active_fun=rnd_search_cv.best_params_["active_fun"],
    epochs=10,  # Fixed training parameter
    #batch_size=32,  # Optional
    verbose=0  # Optional
)

In [21]:
my_model

KerasRegressor(
	model=<function build_model at 0x00000269B4372B80>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	n_hidden=3
	n_neurons=79
	learning_rate=0.013489486879011069
	shape=[8]
	active_fun=relu
)